In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub
import tensorflow_text as text
df_stock = pd.read_csv("maruti_suzuki/MARUTI.NS.csv",
                 parse_dates = ["Date"],
                 index_col = ["Date"])
df_posts = pd.read_excel("maruti_suzuki/maruti_suzuki_final_posts.xlsx")
# removing the unnecessary columns
df_posts.drop(["Unnamed: 0"], axis=1,inplace=True)
# removing spam posts
df_posts = df_posts[df_posts.Spam==0.0]
df_posts.drop(["Spam"],axis=1,inplace=True)
# sliding a window of 7 days and adding all the TIs
from stock_helper import prepare_data
x,y = prepare_data(df_stock)
final_x = x[np.datetime64("2021-11-13"):]
final_y = y[np.datetime64("2021-11-13"):]
# reversing the posts data
df_posts = df_posts[::-1]
final_posts = df_posts[7:]
# loading the sentiment analysis model
sent_model = tf.keras.models.load_model("final_bert")
# removing duplicates from the data
final_posts.drop_duplicates(subset=['Messages'])
# calculating the sentiments score
sentiments = []
prev = np.datetime64("2015-11-12 21:31:26")
for i in final_y.index:
    total=0
    cnt=0
    for j in final_posts.itertuples():
        _,msg,time = j
        if np.datetime64(time)<np.datetime64(i) and np.datetime64(time)>prev:
            total += tf.squeeze(sent_model.predict([msg])).numpy()
            cnt+=1
    prev = np.datetime64(i)
    if(cnt==0):
        sentiments.append(0)
    else:
        sentiments.append(total/cnt)
# getting indices where sentiments score is 0
zero_index = []
for i,j in enumerate(sentiments):
    if(j==0):
        zero_index.append(i)
# removing all the zero values indices
sentiments = np.delete(sentiments,zero_index)
final_x_zeros = final_x.copy()
final_y_zeros = final_y.copy()
final_y_zeros = final_y_zeros.to_frame()
final_x_zeros['removal_assist'] = np.arange(0,len(final_x),1)
final_y_zeros['removal_assist'] = np.arange(0,len(final_x),1)
final_y_zeros = final_y_zeros[final_y_zeros.removal_assist.isin(zero_index)==False]
final_x_zeros = final_x_zeros[final_x_zeros.removal_assist.isin(zero_index)==False]
# removing the added helper column
final_x_zeros.drop(["removal_assist"], axis=1,inplace=True)
final_y_zeros.drop(["removal_assist"], axis=1, inplace=True)

In [2]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
final_x_zeros_scaled = mms.fit_transform(final_x_zeros)
final_x_zeros_scaled

array([[0.41980076, 0.19144392, 0.17350197, ..., 0.57974905, 0.70709467,
        0.54999995],
       [0.36957264, 0.20892048, 0.2366209 , ..., 0.60375047, 0.7176312 ,
        0.54999995],
       [0.2905264 , 0.23642635, 0.33595848, ..., 0.5883486 , 0.6303451 ,
        0.54999995],
       ...,
       [0.95180225, 0.9424734 , 0.94618464, ..., 0.7914287 , 0.7590643 ,
        0.45      ],
       [0.96364975, 0.9706726 , 0.98105717, ..., 0.73870564, 0.7252364 ,
        0.45      ],
       [1.        , 1.        , 1.0000005 , ..., 0.668807  , 0.68463016,
        0.45      ]], dtype=float32)

In [3]:
k = final_y_zeros.max()
final_y_zeros= final_y_zeros/k

In [7]:
model_nbeats = tf.keras.models.load_model("nbeats_maruti suzuki")
model_nbeats.evaluate(final_x_zeros[50:],final_y_zeros[50:]*k)

1/1 [==============================] - 1s 678ms/step - loss: 310.5994 - mae: 310.5994 - mse: 127680.1328


[310.599365234375, 310.599365234375, 127680.1328125]

In [6]:
tf.random.set_seed(42)
inputs = tf.keras.layers.Input(shape=(71))
x = tf.keras.layers.Lambda(lambda x : tf.expand_dims(x,axis=1))(inputs)
x = tf.keras.layers.LSTM(128,return_sequences=True,dropout=0.1)(x)
x = tf.keras.layers.LSTM(128,return_sequences=True, dropout=0.1)(x)
x = tf.keras.layers.LSTM(128,return_sequences=True,dropout=0.1)(x)
x = tf.keras.layers.LSTM(128,return_sequences=True, dropout=0.1)(x)
x = tf.keras.layers.LSTM(128)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.models.Model(inputs = inputs, outputs = output)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 71)]              0         
                                                                 
 lambda_1 (Lambda)           (None, 1, 71)             0         
                                                                 
 lstm_3 (LSTM)               (None, 1, 128)            102400    
                                                                 
 lstm_4 (LSTM)               (None, 1, 128)            131584    
                                                                 
 lstm_5 (LSTM)               (None, 1, 128)            131584    
                                                                 
 lstm_6 (LSTM)               (None, 1, 128)            131584    
                                                                 
 lstm_7 (LSTM)               (None, 128)               1315

In [8]:
model.compile(loss="mae",optimizer = tf.keras.optimizers.Adam(), metrics=["mae","mse"])
history = model.fit(final_x_zeros_scaled[:50],final_y_zeros[:50],
                    epochs = 1000,
                    validation_data = (final_x_zeros_scaled[50:], final_y_zeros[50:]),
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                        patience=200, 
                                                        restore_best_weights=True),
                      tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", 
                                                           patience=100, 
                                                           verbose=1)])

Epoch 1/1000
2/2 [==============================] - 7s 1s/step - loss: 0.8524 - mae: 0.8524 - mse: 0.7285 - val_loss: 0.9555 - val_mae: 0.9555 - val_mse: 0.9131 - lr: 0.0010
Epoch 2/1000
2/2 [==============================] - 0s 57ms/step - loss: 0.8414 - mae: 0.8414 - mse: 0.7098 - val_loss: 0.9426 - val_mae: 0.9426 - val_mse: 0.8887 - lr: 0.0010
Epoch 3/1000
2/2 [==============================] - 0s 57ms/step - loss: 0.8286 - mae: 0.8286 - mse: 0.6885 - val_loss: 0.9260 - val_mae: 0.9260 - val_mse: 0.8578 - lr: 0.0010
Epoch 4/1000
2/2 [==============================] - 0s 56ms/step - loss: 0.8127 - mae: 0.8127 - mse: 0.6623 - val_loss: 0.9028 - val_mae: 0.9028 - val_mse: 0.8153 - lr: 0.0010
Epoch 5/1000
2/2 [==============================] - 0s 49ms/step - loss: 0.7908 - mae: 0.7908 - mse: 0.6271 - val_loss: 0.8673 - val_mae: 0.8673 - val_mse: 0.7525 - lr: 0.0010
Epoch 6/1000
2/2 [==============================] - 0s 52ms/step - loss: 0.7598 - mae: 0.7598 - mse: 0.5787 - val_loss: 0.

2/2 [==============================] - 0s 34ms/step - loss: 0.0296 - mae: 0.0296 - mse: 0.0013 - val_loss: 0.0471 - val_mae: 0.0471 - val_mse: 0.0024 - lr: 0.0010
Epoch 48/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0256 - mae: 0.0256 - mse: 0.0012 - val_loss: 0.0398 - val_mae: 0.0398 - val_mse: 0.0018 - lr: 0.0010
Epoch 49/1000
2/2 [==============================] - 0s 36ms/step - loss: 0.0282 - mae: 0.0282 - mse: 0.0012 - val_loss: 0.0301 - val_mae: 0.0301 - val_mse: 0.0011 - lr: 0.0010
Epoch 50/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0317 - mae: 0.0317 - mse: 0.0015 - val_loss: 0.0550 - val_mae: 0.0550 - val_mse: 0.0032 - lr: 0.0010
Epoch 51/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0211 - mae: 0.0211 - mse: 8.1507e-04 - val_loss: 0.0695 - val_mae: 0.0695 - val_mse: 0.0050 - lr: 0.0010
Epoch 52/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.0295 - mae: 0.0295 - mse: 0.0013 - val_loss: 0.04

Epoch 93/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.0193 - mae: 0.0193 - mse: 5.5887e-04 - val_loss: 0.0533 - val_mae: 0.0533 - val_mse: 0.0030 - lr: 0.0010
Epoch 94/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0172 - mae: 0.0172 - mse: 5.5194e-04 - val_loss: 0.0727 - val_mae: 0.0727 - val_mse: 0.0054 - lr: 0.0010
Epoch 95/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.0240 - mae: 0.0240 - mse: 0.0011 - val_loss: 0.0593 - val_mae: 0.0593 - val_mse: 0.0037 - lr: 0.0010
Epoch 96/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0207 - mae: 0.0207 - mse: 6.6825e-04 - val_loss: 0.0465 - val_mae: 0.0465 - val_mse: 0.0023 - lr: 0.0010
Epoch 97/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0215 - mae: 0.0215 - mse: 7.3325e-04 - val_loss: 0.0606 - val_mae: 0.0606 - val_mse: 0.0038 - lr: 0.0010
Epoch 98/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0199 - mae: 0.0199 - ms

2/2 [==============================] - 0s 33ms/step - loss: 0.0183 - mae: 0.0183 - mse: 5.3366e-04 - val_loss: 0.0536 - val_mae: 0.0536 - val_mse: 0.0030 - lr: 1.0000e-04
Epoch 138/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0164 - mae: 0.0164 - mse: 4.4999e-04 - val_loss: 0.0498 - val_mae: 0.0498 - val_mse: 0.0026 - lr: 1.0000e-04
Epoch 139/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0176 - mae: 0.0176 - mse: 4.7536e-04 - val_loss: 0.0493 - val_mae: 0.0493 - val_mse: 0.0026 - lr: 1.0000e-04
Epoch 140/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.0183 - mae: 0.0183 - mse: 5.3592e-04 - val_loss: 0.0504 - val_mae: 0.0504 - val_mse: 0.0027 - lr: 1.0000e-04
Epoch 141/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0177 - mae: 0.0177 - mse: 4.9497e-04 - val_loss: 0.0536 - val_mae: 0.0536 - val_mse: 0.0030 - lr: 1.0000e-04
Epoch 142/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0203 - m

2/2 [==============================] - 0s 33ms/step - loss: 0.0190 - mae: 0.0190 - mse: 6.9299e-04 - val_loss: 0.0560 - val_mae: 0.0560 - val_mse: 0.0033 - lr: 1.0000e-04
Epoch 182/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.0157 - mae: 0.0157 - mse: 4.2085e-04 - val_loss: 0.0540 - val_mae: 0.0540 - val_mse: 0.0030 - lr: 1.0000e-04
Epoch 183/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0194 - mae: 0.0194 - mse: 6.3854e-04 - val_loss: 0.0512 - val_mae: 0.0512 - val_mse: 0.0027 - lr: 1.0000e-04
Epoch 184/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0158 - mae: 0.0158 - mse: 3.8388e-04 - val_loss: 0.0507 - val_mae: 0.0507 - val_mse: 0.0027 - lr: 1.0000e-04
Epoch 185/1000
2/2 [==============================] - 0s 38ms/step - loss: 0.0186 - mae: 0.0186 - mse: 4.9815e-04 - val_loss: 0.0518 - val_mae: 0.0518 - val_mse: 0.0028 - lr: 1.0000e-04
Epoch 186/1000
2/2 [==============================] - 0s 38ms/step - loss: 0.0171 - m

2/2 [==============================] - 0s 33ms/step - loss: 0.0173 - mae: 0.0173 - mse: 4.7813e-04 - val_loss: 0.0546 - val_mae: 0.0546 - val_mse: 0.0031 - lr: 1.0000e-04
Epoch 226/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0145 - mae: 0.0145 - mse: 3.4687e-04 - val_loss: 0.0539 - val_mae: 0.0539 - val_mse: 0.0030 - lr: 1.0000e-04
Epoch 227/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0195 - mae: 0.0195 - mse: 6.1390e-04 - val_loss: 0.0516 - val_mae: 0.0516 - val_mse: 0.0028 - lr: 1.0000e-04
Epoch 228/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0158 - mae: 0.0158 - mse: 4.3032e-04 - val_loss: 0.0504 - val_mae: 0.0504 - val_mse: 0.0027 - lr: 1.0000e-04
Epoch 229/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0170 - mae: 0.0170 - mse: 4.4890e-04 - val_loss: 0.0493 - val_mae: 0.0493 - val_mse: 0.0025 - lr: 1.0000e-04
Epoch 230/1000
2/2 [==============================] - 0s 35ms/step - loss: 0.0189 - m

In [9]:
model.compile(loss="mae",optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=["mae","mse"])
history = model.fit(final_x_zeros_scaled[:50],final_y_zeros[:50],
                    epochs = 2000,
                    initial_epoch = history.epoch[-1],
                    validation_data = (final_x_zeros_scaled[50:], final_y_zeros[50:]),
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                        patience=200, 
                                                        restore_best_weights=True),
                      tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", 
                                                           patience=100, 
                                                           verbose=1)])

Epoch 234/2000
2/2 [==============================] - 5s 1s/step - loss: 0.0346 - mae: 0.0346 - mse: 0.0019 - val_loss: 0.0349 - val_mae: 0.0349 - val_mse: 0.0014 - lr: 1.0000e-04
Epoch 235/2000
2/2 [==============================] - 0s 50ms/step - loss: 0.0368 - mae: 0.0368 - mse: 0.0019 - val_loss: 0.0271 - val_mae: 0.0271 - val_mse: 9.3354e-04 - lr: 1.0000e-04
Epoch 236/2000
2/2 [==============================] - 0s 52ms/step - loss: 0.0350 - mae: 0.0350 - mse: 0.0017 - val_loss: 0.0139 - val_mae: 0.0139 - val_mse: 3.6692e-04 - lr: 1.0000e-04
Epoch 237/2000
2/2 [==============================] - 0s 51ms/step - loss: 0.0329 - mae: 0.0329 - mse: 0.0016 - val_loss: 0.0102 - val_mae: 0.0102 - val_mse: 2.2730e-04 - lr: 1.0000e-04
Epoch 238/2000
2/2 [==============================] - 0s 54ms/step - loss: 0.0283 - mae: 0.0283 - mse: 0.0013 - val_loss: 0.0099 - val_mae: 0.0099 - val_mse: 2.1108e-04 - lr: 1.0000e-04
Epoch 239/2000
2/2 [==============================] - 0s 45ms/step - loss: 0

2/2 [==============================] - 0s 32ms/step - loss: 0.0237 - mae: 0.0237 - mse: 8.2002e-04 - val_loss: 0.0398 - val_mae: 0.0398 - val_mse: 0.0018 - lr: 1.0000e-04
Epoch 279/2000
2/2 [==============================] - 0s 33ms/step - loss: 0.0269 - mae: 0.0269 - mse: 0.0012 - val_loss: 0.0365 - val_mae: 0.0365 - val_mse: 0.0015 - lr: 1.0000e-04
Epoch 280/2000
2/2 [==============================] - 0s 33ms/step - loss: 0.0288 - mae: 0.0288 - mse: 0.0014 - val_loss: 0.0335 - val_mae: 0.0335 - val_mse: 0.0013 - lr: 1.0000e-04
Epoch 281/2000
2/2 [==============================] - 0s 32ms/step - loss: 0.0247 - mae: 0.0247 - mse: 9.3193e-04 - val_loss: 0.0314 - val_mae: 0.0314 - val_mse: 0.0012 - lr: 1.0000e-04
Epoch 282/2000
2/2 [==============================] - 0s 33ms/step - loss: 0.0266 - mae: 0.0266 - mse: 0.0013 - val_loss: 0.0304 - val_mae: 0.0304 - val_mse: 0.0011 - lr: 1.0000e-04
Epoch 283/2000
2/2 [==============================] - 0s 32ms/step - loss: 0.0278 - mae: 0.0278 -

Epoch 323/2000
2/2 [==============================] - 0s 32ms/step - loss: 0.0251 - mae: 0.0251 - mse: 0.0012 - val_loss: 0.0367 - val_mae: 0.0367 - val_mse: 0.0015 - lr: 1.0000e-04
Epoch 324/2000
2/2 [==============================] - 0s 33ms/step - loss: 0.0288 - mae: 0.0288 - mse: 0.0014 - val_loss: 0.0444 - val_mae: 0.0444 - val_mse: 0.0021 - lr: 1.0000e-04
Epoch 325/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.0252 - mae: 0.0252 - mse: 9.5424e-04 - val_loss: 0.0490 - val_mae: 0.0490 - val_mse: 0.0026 - lr: 1.0000e-04
Epoch 326/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.0233 - mae: 0.0233 - mse: 7.6413e-04 - val_loss: 0.0511 - val_mae: 0.0511 - val_mse: 0.0028 - lr: 1.0000e-04
Epoch 327/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.0236 - mae: 0.0236 - mse: 8.3384e-04 - val_loss: 0.0544 - val_mae: 0.0544 - val_mse: 0.0031 - lr: 1.0000e-04
Epoch 328/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.0

2/2 [==============================] - 0s 40ms/step - loss: 0.0236 - mae: 0.0236 - mse: 7.9526e-04 - val_loss: 0.0504 - val_mae: 0.0504 - val_mse: 0.0027 - lr: 1.0000e-05
Epoch 367/2000
2/2 [==============================] - 0s 39ms/step - loss: 0.0267 - mae: 0.0267 - mse: 0.0010 - val_loss: 0.0500 - val_mae: 0.0500 - val_mse: 0.0027 - lr: 1.0000e-05
Epoch 368/2000
2/2 [==============================] - 0s 36ms/step - loss: 0.0231 - mae: 0.0231 - mse: 8.5197e-04 - val_loss: 0.0491 - val_mae: 0.0491 - val_mse: 0.0026 - lr: 1.0000e-05
Epoch 369/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.0225 - mae: 0.0225 - mse: 8.0544e-04 - val_loss: 0.0484 - val_mae: 0.0484 - val_mse: 0.0025 - lr: 1.0000e-05
Epoch 370/2000
2/2 [==============================] - 0s 33ms/step - loss: 0.0215 - mae: 0.0215 - mse: 7.8763e-04 - val_loss: 0.0481 - val_mae: 0.0481 - val_mse: 0.0025 - lr: 1.0000e-05
Epoch 371/2000
2/2 [==============================] - 0s 32ms/step - loss: 0.0230 - mae: 

2/2 [==============================] - 0s 35ms/step - loss: 0.0269 - mae: 0.0269 - mse: 0.0012 - val_loss: 0.0454 - val_mae: 0.0454 - val_mse: 0.0022 - lr: 1.0000e-05
Epoch 411/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.0279 - mae: 0.0279 - mse: 0.0011 - val_loss: 0.0456 - val_mae: 0.0456 - val_mse: 0.0022 - lr: 1.0000e-05
Epoch 412/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.0240 - mae: 0.0240 - mse: 0.0010 - val_loss: 0.0459 - val_mae: 0.0459 - val_mse: 0.0023 - lr: 1.0000e-05
Epoch 413/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.0263 - mae: 0.0263 - mse: 9.4548e-04 - val_loss: 0.0463 - val_mae: 0.0463 - val_mse: 0.0023 - lr: 1.0000e-05
Epoch 414/2000
2/2 [==============================] - 0s 35ms/step - loss: 0.0233 - mae: 0.0233 - mse: 9.1746e-04 - val_loss: 0.0467 - val_mae: 0.0467 - val_mse: 0.0023 - lr: 1.0000e-05
Epoch 415/2000
2/2 [==============================] - 0s 34ms/step - loss: 0.0268 - mae: 0.0268 -

In [10]:
preds = tf.squeeze(model.predict(final_x_zeros_scaled[50:])).numpy()
preds = preds*k.values[0]
preds

array([8506.652, 8530.543, 8552.735, 8562.858, 8570.454, 8602.898,
       8630.671, 8642.096, 8650.431, 8694.767, 8715.952, 8726.964],
      dtype=float32)

In [12]:
mae = sum(abs(preds-(tf.squeeze(final_y_zeros[50:].to_numpy()).numpy())*k.values[0]))
mae

1062.3447265625

In [16]:
(tf.squeeze(final_y_zeros[50:].to_numpy()).numpy())*k.values[0]

array([8820.2 , 8550.95, 8597.3 , 8559.4 , 8515.25, 8593.65, 8550.55,
       8511.65, 8599.45, 8949.45, 8805.45, 8737.15], dtype=float32)

In [17]:
model.save("LSTM_maruti suzuki")

INFO:tensorflow:Assets written to: LSTM_maruti suzuki\assets


INFO:tensorflow:Assets written to: LSTM_maruti suzuki\assets
